# Sequence chat - extract data, draw image, and summarize




## Import libaries and prepare the API credential

In [1]:
import tempfile
import os
import subprocess

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## Create a temporary folder to execute code


In [2]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)

subprocess.run(["cp", "population.csv", temp_dir.name+"/"])

os.listdir(temp_dir.name)

/tmp/tmpf53pxaix


['population.csv']

## Agent 1: code executor, who can run python code locally

In [3]:
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

## Agent 2: data scientist, who can write python code

In [4]:
data_scientist_system_message = """You are a data scientist, and you can solve problems by writing python code. 
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when the anser is verified.
Some of the tasks are about analyzing a csv file. If you don't know the structure of the csv file, you may provide code to the users to extract the csv structure, and use the feedback from the users to generate the csv analysis code. 
"""

data_scientist = ConversableAgent(
    "data_scientist_agent",
    system_message = data_scientist_system_message,
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

In [5]:
from tools.tools import second_last_msg

nested_chats = [
    {
        "recipient": data_scientist,
        "max_turns": 8,
        "summary_method": second_last_msg
    }
]

code_executor.register_nested_chats(
    nested_chats, 
    trigger = lambda sender: sender not in [data_scientist]
)



In [6]:
quiz_master = ConversableAgent(
    "quiz_master",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS"
)

In [7]:
quiz_master.initiate_chat(
    code_executor,
    message = "can you tell me the population norway the end of each decade from 1950 to 2020? I have a population.csv file, and you can find valuable data there",
    max_turns = 1
)

quiz_master (to code_executor_agent):

can you tell me the population norway the end of each decade from 1950 to 2020? I have a population.csv file, and you can find valuable data there

--------------------------------------------------------------------------------


Provide feedback to quiz_master. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
code_executor_agent (to data_scientist_agent):

can you tell me the population norway the end of each decade from 1950 to 2020? I have a population.csv file, and you can find valuable data there

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

To assist you with finding the population of Norway at the end of each decade from 1950 to 2020, let's first inspect the `population.csv` file to understand its structure. We'll read the CSV file and print the first few rows to get an idea of how the data is organized.

Here's the plan:
1. Read and print the first few rows of the `population.csv` file to understand its structure.
2. Based 

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 0 (execution succeeded)
Code output:   Country name  ...  population__all__95_99__records
0  Afghanistan  ...                                8
1  Afghanistan  ...                                9
2  Afghanistan  ...                               10
3  Afghanistan  ...                               11
4  Afghanistan  ...                               11

[5 rows x 40 columns]


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

Based on the structure of the CSV file, it appears that the columns include population records segmented across various years or age groups (assuming the columns names like `population__all__95_99__records`). We need to determine the relevant columns that indicate populatio

...

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 0 (execution succeeded)
Code output:        Year  Population
12384  1950     3281428
12394  1960     3581521
12404  1970     3875550
12414  1980     4085778
12424  1990     4241637
12434  2000     4491204
12444  2010     4889747
12454  2020     5379836


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

The population of Norway at the end of each decade from 1950 to 2020 is as follows:

- 1950: 3,281,428
- 1960: 3,581,521
- 1970: 3,875,550
- 1980: 4,085,778
- 1990: 4,241,637
- 2000: 4,491,204
- 2010: 4,889,747
- 2020: 5,379,836

This information has been verified based on the extracted data from the CSV file. 

TERMINATE

--------------------------------------------------------------------------

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.
code_executor_agent (to quiz_master):

exitcode: 0 (execution succeeded)
Code output:        Year  Population
12384  1950     3281428
12394  1960     3581521
12404  1970     3875550
12414  1980     4085778
12424  1990     4241637
12434  2000     4491204
12444  2010     4889747
12454  2020     5379836


--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'can you tell me the population norway the end of each decade from 1950 to 2020? I have a population.csv file, and you can find valuable data there', 'role': 'assistant'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output:        Year  Population\n12384  1950     3281428\n12394  1960     3581521\n12404  1970     3875550\n12414  1980     4085778\n12424  1990     4241637\n12434  2000     4491204\n12444  2010     4889747\n12454  2020     5379836\n', 'role': 'user'}], summary='exitcode: 0 (execution succeeded)\nCode output:        Year  Population\n12384  1950     3281428\n12394  1960     3581521\n12404  1970     3875550\n12414  1980     4085778\n12424  1990     4241637\n12434  2000     4491204\n12444  2010     4889747\n12454  2020     5379836\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

## Exercise: how to implement branch (if-else)

The quiz_master must always start with the magic word "show me the code:", otherwise, the code executor will not bother the data scientist.